In [1]:
# Instalar Altair y las librerías necesarias para la exportación de imágenes.
!pip install altair
!pip install altair_saver vl-convert-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 33.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import altair as alt

# Cargar archivo CSV
df = pd.read_csv("Florencia_database_limpia 2.csv", encoding='latin-1', sep=';')

# Corregir nombre de columna con carácter especial
df = df.rename(columns={'A\x96o': 'Año'})

# Filtrar solo categorías escolares
df_escolar = df[df['Categoria'].str.contains('Escolar', case=False, na=False)]

# Seleccionar solo las tres pruebas de salto relevantes
saltos = ['Salto Largo', 'Salto Alto', 'Salto Garrocha']
df_saltos = df_escolar[df_escolar['Prueba'].isin(saltos)]

# Extraer el sexo desde la categoría
df_saltos['Sexo'] = df_saltos['Categoria'].apply(
    lambda x: 'Masculino' if 'Masculino' in x else 'Femenino' if 'Femenino' in x else 'Otro'
)

# Asegurar que el año sea numérico
df_saltos['Año'] = pd.to_numeric(df_saltos['Año'], errors='coerce')

# Contar atletas únicos por prueba, sexo y año
participacion = (
    df_saltos.groupby(['Año', 'Prueba', 'Sexo'])['Nombre Atleta']
    .nunique()
    .reset_index()
    .rename(columns={'Nombre Atleta': 'Atletas Únicos'})
)

# Crear gráfico de barras con facetas por año
chart = (
    alt.Chart(participacion)
    .mark_bar()
    .encode(
        x=alt.X('Prueba:N', title='Prueba de Salto'),
        y=alt.Y('Atletas Únicos:Q', title='Número de Atletas Únicos'),
        color=alt.Color('Sexo:N', title='Sexo', scale=alt.Scale(scheme='tableau10')),
        column=alt.Column('Año:O', title='Año'),
        tooltip=['Año', 'Prueba', 'Sexo', 'Atletas Únicos']
    )
    .properties(
        title='Participación de Atletas Únicos por Prueba, Sexo y Año (Categorías Escolares)',
        width=180,
        height=350
    )
)

chart.show()


alt.Chart(...)